In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset

# Contoh data (bisa diganti dengan data real dari scraping komentar)
data = {
    'comment': [
        'Produk ini sangat bagus, saya suka!',
        'Pelayanan sangat buruk, tidak akan beli lagi!',
        'Biasa saja, tidak terlalu bagus dan tidak terlalu buruk.',
        'Sangat direkomendasikan, kualitas terbaik!',
        'Saya kecewa dengan produk ini, tidak sesuai deskripsi.'
    ],
    'label': [1, 0, 2, 1, 0]  # 1: Positif, 0: Negatif, 2: Netral
}
df = pd.DataFrame(data)

# Encoding label
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Tokenisasi sederhana
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['comment']).toarray()
y = df['label'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# Model Neural Network
class SentimentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SentimentModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Inisialisasi model
input_size = X_train.shape[1]
model = SentimentModel(input_size=input_size, hidden_size=16, output_size=3)

# Loss dan optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 10
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Evaluasi
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')


Epoch 1/10, Loss: 1.1195
Epoch 2/10, Loss: 1.1031
Epoch 3/10, Loss: 1.0931
Epoch 4/10, Loss: 1.0534
Epoch 5/10, Loss: 1.0562
Epoch 6/10, Loss: 1.0462
Epoch 7/10, Loss: 1.0697
Epoch 8/10, Loss: 1.0353
Epoch 9/10, Loss: 1.0056
Epoch 10/10, Loss: 0.9983
Accuracy: 0.00%
